# Temporal Analysis, manosphere

In [1]:
from helpers import jaccard, populate_bin_with_channel, bins_y_o, bins_t_o, bin_to_df
import pandas as pd
import pickle
import json
import gzip

SRC = "/dlabdata1/manosphere-rad/"

# aux functions & vars
def helper_df(df, shenanigans=True):
    df["idx_str"] = bins_t_o
    df_years = df.tail(11)
    if shenanigans:
        df_years["x+y"] = df_years["x"] + df_years["y"] - df_years["intersection"] 
        df_years["xyz_p"] = df_years["intersection"] / df_years["x+y"]
        df_years["intersection/x"] = df_years["intersection"] / df_years["x"]
    return df_years

cats_to_control = ["center", "left-center", "right-center", "left", "right"]
cats_all = list(df_sources.Category.value_counts().keys())
cats = [cat for cat in cats_all if cat not in cats_to_control]
cats.append("Control")
cat_man = ['MGTOW', 'Incel', 'MRA', "PUA"]
cats.append('Manosphere')
df_sources = pd.read_csv(SRC + "sources_final_trimmed.csv")

## Common Commenting Users Along the Years

Computation pipeline

In [2]:
# Read channels
with gzip.open(SRC+"channels_youtube.json.gz", 'rb') as f:
    channel_dict = json.load(f)

In [3]:
# Create bins and populate them with channels
bins_dict = {}
for cat in cats:
    bins_dict[cat] = {b:set() for b in bins_y_o}

for cat in cats_all:
    final_cat = cat if cat in cats else "Control"
    for channel in df_sources.loc[df_sources["Category"] == cat, "Id"]:
        populate_bin_with_channel(channel, channel_dict, bins_dict[final_cat])
        if cat in cat_man:
            populate_bin_with_channel(channel, channel_dict, bins_dict['Manosphere'])
            

In [4]:
# Creates shifted bins
bins_sl_dict = {}

for cat, bin_dict in bins_dict.items():
    bin_sl_dict = {b:set() for b in bins_y_o}
    for idx in range(len(list(bin_dict.keys())) - 1):
        shifted = sorted(bin_dict.keys())[idx+1]
        actual = sorted(bin_dict.keys())[idx]
        bin_sl_dict[shifted] = bin_dict[actual]
    bins_sl_dict[cat] = bin_sl_dict

In [5]:
# Create bins intersection given the specified function, here, `jaccard`
df_inter = {}
    
for cat1 in cats:
    for cat2 in cats:
        bins2 = bins_dict if cat1 != cat2 else bins_sl_dict
        df_inter[(cat1, cat2)] = helper_df(bin_to_df(bins_y_o, bins_dict[cat1], bins2[cat2], jaccard))

with open("../helper_files/intersections.pickle", "wb") as f:
    pickle.dump(df_inter, f)